## 二分探索
O(log n)

最小値の最大化、最大値の最小化などの場合にうまく使えることがある   
また、全組み合わせを試すことができない場合（組み合わせ多すぎて間に合わない、そもそも組み合わせに落とし込めない）に、条件を満たすかどうか、適当な値ずつ範囲を狭めて答えを絞っていく方法が考えられるが、その時に使える。  
一度のループで範囲が半分ずつになっていく。

In [ ]:
"""
POJ 1064 Cable master


同じ長さの紐をK本切り出す
・最長でもmax(L)
・長さaを作ると決めたとき、Liから切り出せるのは Li//a 本

aをどの値にすればいいか求める
案１：１～max(L)まで１ずつ増やして確かめる：max(L)*Nより間に合わない
案２：二分探索で決める
"""

N = int(input())
K = int(input())
L = list(map(float, input().split()))

ML= max(L)
lo = 0
hi = ML
# 配列の位置の絞り込みとかなら、while hi - lo > 1　みたいな条件にするが、
# 今回は mid がいい感じの値に（小数第2位まで出力）落ち着けばいいのでそこそこの回数繰り返せばいい
# 一回ごとに範囲が半分になるので、30回もやれば2**30分の1になる
for _ in range(300):
    mid = (lo+hi)/2
    # mid分切り出したときに何本作れるか
    cnt = 0
    for l in L:
        cnt += l // mid
    if cnt >= K:
        lo = mid
    else:
        hi = mid

print(mid)


"""
4
11
8.02 7.43 4.57 5.39
2.005
"""

In [ ]:
"""
POJ 2456 Agressive cows

試行を繰り返して求めるのは難しい（というか無理？）
愚直な方法として
案１：あらゆる牛の配置法を全て試して、それぞれの最短距離の中から最大値を選ぶ：間に合うわけがない
案２：距離１ずつ離して配置できるか試す→２ずつ離して置けるか試す→...　を繰り返して、置けなくなった時の手前の回の距離が最大の最小距離
　　→これも間に合わない（牛の数やら小屋の配置に比例するので）
案２の方法は、１ずつ範囲が絞り込まれてるのが無駄というかボトルネック。一回の施行でもっと答えを絞っていきたい
　　→　二分探索
"""


N = int(input())
M = int(input())
X = list(map(int, input().split()))

X.sort()
lo = 1
hi = X[-1]
mid = (lo+hi)//2
while hi-lo > 1:
    #print(lo, hi, mid)
    cnt = 1
    curr = X[0]
    idx_next = 1
    while idx_next <= len(X)-1:
        if mid <= X[idx_next] - curr:
            cnt += 1
            curr = X[idx_next]
        idx_next += 1
        if cnt == M:
            lo = mid
            break
        elif cnt < M and idx_next >= len(X)-1:
            hi = mid
            break
    mid = (lo+hi)//2
print(mid)


"""
5
3
1 2 8 4 9

"""

In [ ]:
"""
平均最大化

一見 v/w でソートして大きいものから選べばいいようにも思えるが無理。
案１：ｋ個選ぶ方法をすべてためして重さ当たりの価値が最大の物を選ぶ：間に合わない
案２：単位当たり重さが0以上→0.1以上→ ... と試していき、重さ当たり価値の最大を選んでいく：無理（刻みが分からない、時間がかかる）

単位当たり重さ：Σ(v) / Σ(w) >= x 
なるｘについてｘが最大のものを見つけたい
→ Σ(v) - x * Σ(w) >= 0
→ Σ(v - w*x) >= 0
これがある品物の平均に対する寄与度みたいなもので、これが成立する、値の大きいものからｋ個選んだ時にそれらの和が0以上なら或る重さ当たり価値xは作ることができる

"""

N = int(input())
K = int(input())
wv = []
for i in range(N):
    wv.append(tuple(map(int, input().split())))



lo = 0
hi = 10**6
mid = (lo+hi)/2.0
# 今回もインデックスを求めるわけではないので、hi-lo>1 でそれっぽい値に落ち着かない
# 適当な回数繰り返すか、変化量が一定以下になったら止めるとかでいいと思う
for _ in range(30):
    y = [0] * N
    for i in range(N):
        y[i] = wv[i][1] - mid*wv[i][0]

    y.sort(reverse=True)
    # 大きいほうからk個取得
    total = sum(y[:K])
    if total > 0:
        lo = mid
    else:
        hi = mid
    mid = (hi+lo)/2.0

print(mid)


"""
3
2
2 2
5 3
2 1

"""

In [ ]:
"""
AtCoder Regular Contest 037 C



小さいほうからK番目の値がX：X-1以下の値がK個未満しかない＆X以下の値がK個以上
→ X以下の値がK個以上ある最小のXを求める
ここでXはAとBの要素の積からできる。

Xをなんか決めてX以下の値がK個以上ある　のXについて最小を求める
AとBをソートしておく
普通にAxBをすべてやり、それがX以下か確かめると間に合わないので、
Ai*Bj <= X -> Bj <= X // Ai
から、各行について上の式を満たすjの個数（＝その行のAxBにおいてX以下となる個数）を二分探索で求め、全行の個数の合計がK以上となる最小のXを探す

pypyじゃないと間に合わない

"""

from bisect import bisect

N,K = map(int, input().split())

A = list(map(int, input().split()))
B = list(map(int, input().split()))

A.sort()
B.sort()


lo = 0
hi = (10**9)**2

while hi - lo > 1:
    mid = (hi+lo)//2
    # 各行にあるX（変数としてはmid）以下の個数を求める
    cnt = 0
    for i in range(N):
        cnt += bisect(B, mid // A[i])
    
    # cntがKを上回るなら、そのX（変数としてはmid）は大きすぎる（Xを10**18とかとればAxBのすべてがX以下になるみたいな）
    if cnt >= K:
        hi = mid
    else:
        lo = mid
    

print(hi)


In [ ]:
"""

AtCoder Beginner Contest 023 D

考察
１：全通り試して最小を求める：無理
２：初期高さ順に割る：正しくない
３：スピード順：正しくない
SとHが大きすぎるのが気になる。
N*Sが10**(9+5)とかになるので、問題的に高さを実際に見ていく感じではなさそう
位置 ＝ 速度 x 時間なので、速度か時間で見る。
速度は固定で与えられるのでこれを軸に評価する方法はよくわからない。
時間を軸として評価できないか？
→ 時間　＝　位置 / 速度

得点をXにすることができるか試す
得点がXとは、全ての風船の中で、一番上にいったやつの高さがXということ。
つまり、全ての風船は高さX以下で割られている。
ある風船について、初期高さがHiで、時間当たり上昇量がSiなので
Hi ＋ Si*t <= Xの領域にいる状態で割られているということ。
これを時間を軸として式変換すると、
t <= (X - Hi) / Si
となり、この時刻のうちにそれぞれの風船が割られる。 

全ての風船について、これを満たすことができる最小のXを探す
同じ時刻に風船が二つ以上破られることがないのでこれで全ケースとおるか心配ではある
ので、上の式の結果を配列に並べてソートして、時刻の重複が起こらないかもみる。


"""


N = int(input())

H = [0]*N
S = [0]*N
for i in range(N):
    H[i], S[i] = map(int, input().split())

# N <= 10**5, S <= 10**9なので一桁多めに取った
hi = 10**15
lo = 0

while hi - lo > 1:
    mid = (hi+lo)//2
    # (mid - H[i])//S[i]までに風船iを割れば良い
    times = [(mid - H[i])/S[i] for i in range(N)]
    # 早く割る必要があるものから並べる
    times.sort()
    # 全ての風船が指定した高さmidに達するまでに割れるかどうか
    flag = True
    # 時刻0、1、2、...1、N-1で風船を割れるので、そのタイミングで間に合っているか
    for i in range(N):
        if times[i] < i:
            flag = False
            break
    # 指定の高さmidに達するまでに風船を割り切れるので、midより高さを低くして再確認
    if flag:
        hi = mid
    # 
    else:
        lo = mid

# hiが条件を満たす最小の高さ
print(hi)

In [ ]:
"""

AtCoder Regular Contest 050 B
赤多め：赤がｘ本、青が１本必要
青多め：赤が１本、青がｙ本必要
全部でK個花束を作ることができるかどうか考える。赤多めをP個、青多めをQ個作るとすると、
赤の必要本数：x*P + 1*Q <= R
青の必要本数：1*P + y*Q <= B
K個作れる場合、K=P+Qを入れると、
赤：(x-1)*P <= R-K -> P <= (R-K)/(x-1)
青：(y-1)*Q <= B-K -> Q <= (B-K)/(y-1)
P+QはK以上でなけらばならないので
K <= P+Q <= (R-K)/(x-1) + (B-K)/(y-1)
みたいな感じ
実際PとQは整数なので // になるが。

マジで難しい
"""

R, B = map(int, input().split())
x, y = map(int, input().split())

# x,y > 2なので10**18個もできることはないけど、大きめに取っておく
hi = 10**18
lo = 0
while hi - lo > 1:
    
    mid = (hi+lo)//2
    #print(hi, lo, mid)
    # 初期のhiを大きめに取ったので、RやBがmid以上かどうかも確認
    if (R-mid) // (x-1) + (B-mid) // (y-1) >= mid and R >= mid and B >= mid:
        lo = mid
    else:
        hi = mid

print(lo)

In [ ]:
"""

AtCoder Beginner Contest 026 D

普通ににぶたんで範囲をしぼっていく
"""

import math

A,B,C = map(int, input().split())

hi = 200.0
lo = 0.0
mid = (hi+lo)/2
while abs(A*mid + B*math.sin(C*math.pi*mid) - 100) > 10**(-6):
    if A*mid + B*math.sin(C*math.pi*mid) > 100:
        hi = mid
    else:
        lo = mid

    mid = (hi+lo)/2

print(mid)



In [ ]:
"""
AtCoder Beginner Contest 034 D

食塩水の濃度がx
→　（各容器内の食塩の総量）/ （各容器内の食塩水の総量）* 100 = x
容器ごとに全量wと濃度pが与えられるので、
Σ(w_i * (p_i/100)) * 100 / Σ(w_i) = x
ある濃度x以上にできるのかを考えると

Σ(w_i * (p_i/100)) * 100 / Σ(w_i) >= x

-> Σ(w_i *p_i - x*w_i) >= 0

これを満たすxの中で最大のものを求める
"""

N,K = map(int, input().split())

wp = [tuple(map(int, input().split())) for _ in range(N)]

hi = 100.0
lo = 0.0
mid = (hi+lo)/2.0

for _ in range(40):
    tmp = [0] * N
    for i in range(N):
        tmp[i] = wp[i][0] * wp[i][1] - mid*wp[i][0]

    tmp.sort(reverse=True)
    total = sum(tmp[:K])
    if total >= 0:
        lo = mid
    else:
        hi = mid
    mid = (hi+lo)/2.0

print(mid)


In [ ]:
"""
AtCoder Beginner Contest 155  D - Pairs

とても難しい
考え方としてはけんちょんさんの解説（https://drken1215.hatenablog.com/entry/2020/04/23/142200）が非常にわかりやすい。
が、これを素直にpythonでやるとTLE（探索の途中でok,ngを判定することができる場合でも、最後まで探索を続けてしまいTLEになる）

なので、いい感じに途中で探索を打ち切れる形にする
また、途中で打ち切るために、判定をis_okのような関数にしておくとやりやすい


考え方としては、
・小さいものからK番目を求めろ
　→　x以下となる物について個数の合計がK個以上となるxについて、最小のxを調べる
に言い換えることができる。
ここで最小の～を探すのに二分探索を使うとよい

ARC-37 Cと発想は同じっぽいが実装がしんどい
"""


from bisect import bisect

N,K = map(int, input().split())
A = list(map(int, input().split()))
A.sort()

cnt_positive = N - bisect(A, 0)
cnt_zero = A.count(0)
cnt_negative = N - cnt_positive - cnt_zero


def is_ok(mid):
    
    if mid < 0:
        cnt_under_mid = 0
        # i=0のときと組み合わせが被らないようにjは1から
        j = 1
        for i in range(cnt_negative):
            a = A[i]
            # A[i]が負なので、A[i] * A[j]は単調減少
            while a * A[j] > mid:
                j += 1
                if j >= N:
                    break
            # あるiについて、A[i] * A[j~N-1] <= mid を満たすペアの数を足す
            cnt_under_mid += (N - j)

            # この時点で積がmid以下になるペアの数がK個以上あれば、そのmidについてそれ以上調べる必要はない
            if cnt_under_mid >= K:
                return True

            # この時点で積がmidのペアの個数がK未満かつ、それ以上mid以下にできる組み合わせがない場合、探索を打ち切る
            if j >= N:
                break

        return False

    # midが0以上の場合、

    else:
        """
        こちらではmidより大きくなる個数を全ペアの個数から引いて考える
        """
        cnt_over_mid = 0
        cnt_duplicate = 0
        r = cnt_negative-1 # A[l]が負のindexで一番右にあるやつ
        for l in range(cnt_negative):
            a = A[l]
            # A[l] * A[r]はrを左に寄せてくるにつれて大きくなる（負 x 負で,右側の負の徐々に絶対値が大きくなるから）
            while a * A[r] <= mid:
                r -= 1
                # lについて、いずれのrをペアにとってもmidを上回らないなら、負の要素についての探索を打ち切る
                if r <= -1:
                    break
            cnt_over_mid += r+1 # 例えばidxでj=2になった時に積がmidを上回るなら、j=0,1,2の三つとの積でmidを超える、みたいな
            # 同じインデックスでペアを作る場合、重複を取り除く
            if r >= l: cnt_duplicate += 1
            # iを右に動かすにつれて、ｊを左に動かさないとmidを超える積ができなくなる（iを右にすると絶対値が小さくなるので、jのほうは絶対値を大きくする必要がある）
            # jをもう動かすことができなくなったら、A[i]が負の場合に、それ以上midを上回るペアを作れないので探索を打ち切る
            if r <= -1:
                break
        
        # Aの要素について、負以外の場合
        cnt_not_positive = cnt_negative + cnt_zero
        l = cnt_not_positive
        # 今度は右から見ていって、対象をlとして、lより右側にある物とペアになればmidを超える、みたいな
        for r in range(N-1, cnt_not_positive-1, -1):
            a = A[r]
            while a * A[l] <= mid:
                # ペアを作って積がmidを超えるまで、lを右にずらしていく
                l += 1
                # これ以上lを右にずらせない場合打ち切り
                if l >= N:
                    break
            # あるrについて、lより右にあるものとペアを作るとmidを超えるので、その個数を足す
            cnt_over_mid += N-l
            # rのペアにrを選んでいる場合は取り除く
            if l <= r:
                cnt_duplicate += 1

            # それ以上lを右にずらせない場合は探索を打ち切る
            if l >= N:
                break
        
        cnt_under_mid = (N*(N-1) // 2) - ((cnt_over_mid - cnt_duplicate) // 2)
        return cnt_under_mid >= K


ok = 10**18 + 1
ng = -1 * 10**18 - 1

while ok - ng > 1:
    mid = (ok + ng) // 2
    if is_ok(mid):
        ok = mid
    else:
        ng = mid

print(ok)

In [ ]:
"""
AtCoder Petrozavodsk Contest 001  C - Vacant Seat


交互に座るので、N/2離れた位置の人が0と同じ性別かどうかわかるはず。
空席が間にあれば、想定と違う性別になるので、二分探索で候補を一回ごとに半分に削っていく

"""



N = int(input())


def query(idx):
    print(idx)
    s = input()
    if s == "Vacant":
        exit()
    
    return s

base = query(0)

def is_ok(mid):
    s = query(mid)
    # 最初にクエリを投げた場所との差が偶数なので、0~mid間で空席がなければ、midの位置の人は同じ性別になるはず
    if mid % 2 == 0:
        return s == base
    # 偶数の差でないので、midの位置には異なる性別の人が座っているはず、
    else:
        return s != base

lo = 0
hi = N
while hi - lo > 1:
    mid = (hi + lo) // 2
    if is_ok(mid):
        lo = mid
    else:
        hi = mid


In [ ]:
"""
AtCoder Regular Contest 043  B - 難易度




二問目と三問目を決め打ってそれぞれで二分探索するとよい。が、N<= 10**5なので間に合わない
二問目を決めたときに３問目の取り方と四問目の取り方が同時にばちっとわかる必要がある。（O(N^2) -> O(N)にしたい。）

二問目を決めたときに、
（１）：その二問目の難易度の半分以下の問題の個数が何個あるか
（２）：その二問目を選んだ時に、３，４問目の取り方が何個あるか
の積をとって、２問目を先頭から最後まで選んでいけばよい

で、（２）を得るには、核問題を三問目として選んだ時の四問目の取り方を先に計算しておいて、
２問目を選んだ時に、３問目として取りうるものを累積でたしていけばよいか？
"""


import bisect

N = int(input())
D = [int(input()) for _ in range(N)]
D.sort()
MOD = 10**9 + 7


# Diを二問目に選んだ時に、一問目として何個選べるか
num_first = [0] * N
for i in range(N):
    idx = bisect.bisect_right(D, D[i] / 2.0)
    num_first[i] = idx

# Diを３問目に選んだ時に、４問目として選びうるものが何個あるか。
num_forth = [0] * N
for i in range(N):
    idx = bisect.bisect_left(D, 2*D[i])
    num_forth[i] = N - idx

# D[i:]でできる、３問目と４問目の組の数
num_pair_34 = [0] * N
for i in range(N):
    if i == 0:
        num_pair_34[N-1-i] = num_forth[N-1-i]
    else:
        num_pair_34[N-1-i] = num_pair_34[N-i] + num_forth[N-1-i]


ans = 0
for i in range(N):
    # D[i]を二問目に選ぶ
    idx = bisect.bisect_left(D, 2*D[i])
    if idx == N:
        continue
    ans += num_first[i] * num_pair_34[idx]
    ans %= MOD

print(ans)
